In [1]:
import csv
import json

import deepl
import numpy as np
import pandas as pd
import wn
from tqdm.notebook import tqdm

In [2]:
auth_key = ""  # Add here your key
translator = deepl.Translator(auth_key)
filename = "data/gaps_translation.csv"

with open("trees_path/path_with_gaps.json", "r") as f:
    data = json.load(f)
data = data[:20]

wordnet = pd.read_json(path_or_buf="data/wordnet_translated_clean.jsonl", lines=True)

In [3]:
def transform_list(input_list):
    return "; ".join([", ".join(group) for group in input_list])


def get_translation(text_data):
    return translator.translate_text(text_data, target_lang="UK").text

In [7]:
header = [
    "PWN",
    "ILI",
    "POS",
    "Gap",
    "DeepL Direct",
    "DeepL Contextualized",
    "Translated Wordnet",
    "Lemmas",
    "Gloss",
    "Hypernyms",
    "Hyponyms",
]
with open(filename, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(header)
    for elem in tqdm(data, total=len(data)):
        pwn = next(iter(elem["path"]))
        for key, value in elem["path"].items():
            if "*" in value["title"]:
                gap = value["title"][1:-1]
                ss = wn.synset(key)
                gloss = ss.definition()
                ili = value["ili"]
                lemmas = ", ".join(ss.lemmas())
                hypernyms = [hypernym.lemmas() for hypernym in ss.hypernyms()]
                hyponyms = [hyponym.lemmas() for hyponym in ss.hyponyms()]
                sentence = f"{lemmas} — {gloss}."

                deepL_direct = get_translation(gap)
                sentence_translation = get_translation(sentence)
                deepL_contextualized = sentence_translation.split(" - ")[0].split(",")[
                    0
                ]
                try:
                    translated_wordnet = wordnet.loc[wordnet.ili == ili].iloc[0][
                        "lemma_raw"
                    ]
                except IndexError:
                    translated_wordnet = np.nan
                gap_data = [
                    key,
                    ili,
                    ss.pos,
                    gap,
                    deepL_direct,
                    deepL_contextualized,
                    translated_wordnet,
                    lemmas,
                    gloss,
                    transform_list(hypernyms),
                    transform_list(hyponyms),
                ]
                writer.writerow(gap_data)

  0%|          | 0/20 [00:00<?, ?it/s]